Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [9]:
# To join Part 1 and Part 2 dataframes, you can use the merge function in Pandas. 
# The common columns between both dataframes are the latitude and longitude values. 
# We'll use these columns as the keys to perform the merge.

import requests
import pandas as pd
import os

def get_bike_stations(city):
    url = f"https://api.citybik.es/v2/networks/{city}"

    try:
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            # Extract station details
            stations = data['network']['stations']

            # Create a list to store station information
            station_data = []
            for station in stations:
                station_name = station['name']
                latitude = station['latitude']
                longitude = station['longitude']
                num_bikes = station['free_bikes']

                # Append station information to the list
                station_data.append({
                    'Station Name': station_name,
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Available Bikes': num_bikes
                })

            # Create a DataFrame from the station data list
            df = pd.DataFrame(station_data)

            return df

        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")

if __name__ == "__main__":
    city_id = "almatybike"
    bike_stations_df = get_bike_stations(city_id)
    bike_stations_df = bike_stations_df.head(10)
    print(bike_stations_df)

def get_foursquare_data():
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": "bar",
        "ll": "43.2391,76.9492", 
        "radius": "1000"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": os.environ["FOURSQUARE_API_KEY"]
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch Foursquare data. Status code: {response.status_code}")
        return None
    
def dataframe_foursquare_data(foursquare_data):
    if foursquare_data is None:
        return None

    businesses = foursquare_data.get("results", [])

    if not businesses:
        print("No businesses found.")
        return None
    
    extracted_data = []
    for business in businesses:
        rating_data = business.get("rating", None)
        business_data = {
            "Name": business.get("name", ""),
            "Categories": ", ".join(category.get("name", "") for category in business.get("categories", [])),
            "Distance": business.get("distance", ""),
            "Latitude": business.get("geocodes", {}).get("main", {}).get("latitude", ""),
            "Longitude": business.get("geocodes", {}).get("main", {}).get("longitude", ""), # we can addbelow details as well
            # "Address": business.get("location", {}).get("formatted_address", ""),
            # "Region": business.get("location", {}).get("region", ""),
            # "Country": business.get("location", {}).get("country", ""),
            # "Cross_Street": business.get("location", {}).get("cross_street", ""),
            # "Link": business.get("link", ""),
            # "Timezone": business.get("timezone", ""),
            # "Rating": rating_data if rating_data is not None else "N/A",  # Adding the rating or "N/A" if not available
        }
        extracted_data.append(business_data)

    # Convert the extracted data to a DataFrame
    df = pd.DataFrame.from_dict(extracted_data)
    return df

if __name__ == "__main__":
    foursquare_data = get_foursquare_data()
    if foursquare_data is not None:
        df = dataframe_foursquare_data(foursquare_data)
        df = df.head(10)
        print(df)

if __name__ == "__main__":
    # Part 1: Get bike stations dataframe
    city_id = "almatybike"
    bike_stations_df = get_bike_stations(city_id)

    # Part 2: Get Foursquare data and create Foursquare dataframe
    foursquare_data = get_foursquare_data()
    if foursquare_data is not None:
        foursquare_df = dataframe_foursquare_data(foursquare_data)

        # Merge the two dataframes based on latitude and longitude columns
        merged_df = pd.merge(bike_stations_df, foursquare_df, on=["Latitude", "Longitude"], how="left")
        merged_df = merged_df.head(10)

        print(merged_df)

                            Station Name   Latitude  Longitude  \
0          ул. Сатпаева - пр. Назарбаева  43.239100  76.949200   
1                  пр. Абая - пр. Достык  43.243218  76.956374   
2        ул.Пушкина - ул.Кабанбай батыра  43.250500  76.954100   
3                       ТРЦ Esentai Mall  43.217743  76.928146   
4         ул.Ходжанова - пр.Аль-Фараби    43.206754  76.913320   
5          пр.Аль-Фараби – ул.Мустафина   43.194999  76.880626   
6  ул.Гагарина, 292 (выше ул. Ескараева)  43.206270  76.898812   
7        ул.Байтурсынова – ул.Габдуллина  43.233657  76.931573   
8          ул.Жандосова – ул.Розыбакиева  43.227652  76.890985   
9             ул.Сатпаева - река Есентай  43.236600  76.921700   

   Available Bikes  
0                3  
1                4  
2                0  
3                8  
4                3  
5                3  
6                6  
7                7  
8                1  
9                5  
                   Name         Categories

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [8]:
# To create a common visualization in Python we need to use the matplotlib and seaborn libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
var = 'Station Name'
data = merged_df[['Available Bikes',var]]
data.plot.scatter(x=var, y='Available Bikes', ylim=(0,10))

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [12]:
import sqlite3
import pandas as pd

# Sample data for the DataFrame
data = {
    'Station Name': [
        'ул. Сатпаева - пр. Назарбаева',
        'пр. Абая - пр. Достык',
        'ул.Пушкина - ул.Кабанбай батыра',
        'ТРЦ Esentai Mall',
        'ул.Ходжанова - пр.Аль-Фараби',
        'пр.Аль-Фараби – ул.Мустафина',
        'ул.Гагарина, 292 (выше ул. Ескараева)',
        'ул.Байтурсынова – ул.Габдуллина',
        'ул.Жандосова – ул.Розыбакиева',
        'ул.Сатпаева - река Есентай'
    ],
    'Latitude': [
        43.239100,
        43.243218,
        43.250500,
        43.217743,
        43.206754,
        43.194999,
        43.206270,
        43.233657,
        43.227652,
        43.236600
    ],
    'Longitude': [
        76.949200,
        76.956374,
        76.954100,
        76.928146,
        76.913320,
        76.880626,
        76.898812,
        76.931573,
        76.890985,
        76.921700
    ],
    'Available Bikes': [4, 4, 0, 8, 3, 4, 6, 7, 1, 6]
}

# Create a DataFrame from the sample data
df = pd.DataFrame(data)

# Define the path to the SQLite database file
db_file = 'D:/courses/Lighthouse/Course/Project_2/my_database.db'

# Create a connection to the database
conn = sqlite3.connect(db_file)

# Store the DataFrame in the database
df.to_sql('bike_stations', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()
print(data)

{'Station Name': ['ул. Сатпаева - пр. Назарбаева', 'пр. Абая - пр. Достык', 'ул.Пушкина - ул.Кабанбай батыра', 'ТРЦ Esentai Mall', 'ул.Ходжанова - пр.Аль-Фараби', 'пр.Аль-Фараби – ул.Мустафина', 'ул.Гагарина, 292 (выше ул. Ескараева)', 'ул.Байтурсынова – ул.Габдуллина', 'ул.Жандосова – ул.Розыбакиева', 'ул.Сатпаева - река Есентай'], 'Latitude': [43.2391, 43.243218, 43.2505, 43.217743, 43.206754, 43.194999, 43.20627, 43.233657, 43.227652, 43.2366], 'Longitude': [76.9492, 76.956374, 76.9541, 76.928146, 76.91332, 76.880626, 76.898812, 76.931573, 76.890985, 76.9217], 'Available Bikes': [4, 4, 0, 8, 3, 4, 6, 7, 1, 6]}


Look at the data before and after the join to validate your data.

In [ ]:
# After joining the data, we can see what POIs nearby bike stations within 1000 meters